In [1]:
import io
import torch

from PIL import Image
import requests
from matplotlib.pyplot import imshow

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageNet

%matplotlib inline
%load_ext autoreload
%autoreload 2